In [ ]:
import cobra
import cobra.test
import pandas as pd
import numpy as np
import Bio.PDB.Polypeptide as prot
import libsbml
import cobrame

In [ ]:
ijo = cobra.io.read_sbml_model('../models/iJO1366.xml')
core = cobra.io.read_sbml_model('../models/ColiCore.xml')
base = cobra.io.read_sbml_model('../models/ColiGS.xml')
pruned = cobra.io.read_sbml_model('../models/ColiPruned.xml')
super_pruned = cobra.io.read_sbml_model('../models/ColiPrunedComp_withoutConsRel.xml')

In [ ]:
pruned.objective = pruned.reactions.Bio_T

In [ ]:
pruned.optimize()

In [ ]:
pruned.summary()

In [ ]:
varner = cobra.io.load_json_model('../models/varner.json')

In [ ]:
varner.reactions.query('transcription')

In [ ]:
def extract_txtl_rxns(model):
    aa_metabs = []
    for aa in prot.aa3:
        aa_metabs += model.metabolites.query(aa.lower())
    aa_rxns = []
    for aa_metab in aa_metabs:
        aa_rxns += aa_metab.reactions
    tx_rxns = model.reactions.query('transcription')
    tl_rxns = model.reactions.query('translation')
    txtl_rxns = list(set(aa_rxns).union(tx_rxns).union(tl_rxns))
    return txtl_rxns
txtl_rxns = extract_txtl_rxns(varner)

In [ ]:
varner.objective = varner.reactions.but_export

In [ ]:
varner.optimize()

In [ ]:
varner.summary()

In [ ]:
def add_enzyme(enz):
    if enz == 'adhE':
        enzs = ['adhE1', 'adhE2']
    else:
        enzs = [enz]
    for enz in enzs:
        rxn = varner.reactions.get_by_id('R_{0}'.format(enz.lower()))
        rxn.add_metabolites(metabolites={varner.metabolites.get_by_id('PROTEIN_{0}'.format(enzs[0])):-1})

In [ ]:
varner.reactions.get_by_id('R_atoB')

In [ ]:
atob.add_metabolites(metabolites={varner.metabolites.PROTEIN_:-1})

In [ ]:
add_enzyme('atoB')

In [ ]:
#add_enzyme('Hbd1')
#add_enzyme('Crt1')
#add_enzyme('Ter1')
add_enzyme('adhE')

In [ ]:
import cobrame

In [ ]:
me = cobrame.MEModel('test')
for met in ijo.metabolites:
    me.add_metabolites(met)
for rxn in ijo.reactions:
    me.add_reaction(rxn)

In [ ]:
# "Translational capacity" of organism
me.global_info['kt'] = 4.5  # (in h-1)scott 2010, RNA-to-protein curve fit
me.global_info['r0'] = 0.087  # scott 2010, RNA-to-protein curve fit
me.global_info['k_deg'] = 1.0/5. * 60.0  # 1/5 1/min 60 min/h # h-1

# Molecular mass of RNA component of ribosome
me.global_info['m_rr'] = 1453. # in kDa

# Average molecular mass of an amino acid
me.global_info['m_aa'] = 109. / 1000.  # in kDa

# Proportion of RNA that is rRNA
me.global_info['f_rRNA'] = .86
me.global_info['m_nt'] = 324. / 1000.  # in kDa
me.global_info['f_mRNA'] = .02

# tRNA associated global information
me.global_info['m_tRNA'] = 25000. / 1000.  # in kDA
me.global_info['f_tRNA'] = .12

# Define the types of biomass that will be synthesized in the model
me.add_biomass_constraints_to_model(["protein_biomass", "mRNA_biomass", "tRNA_biomass", "rRNA_biomass",
                                     "ncRNA_biomass", "DNA_biomass", "lipid_biomass", "constituent_biomass",
                                     "prosthetic_group_biomass", "peptidoglycan_biomass"])

In [ ]:
names = ['atoB', 'Hbd1', 'Crt1', 'Ter1', 'adhE1']
sequences = []
for name in names:
    with open('../genes/{0}.txt'.format(name.lower()), 'r') as f:
        sequences.append(f.read().strip())

In [ ]:
building.create_transcribed_gene(me, 'b','tRNA', 'ATCG')
building.add_transcription_reaction(me, 'TU_b', {'b'}, sequence)
print(me.reactions.transcription_TU_b.reaction)
me.reactions.transcription_TU_b.update()

In [ ]:
from qminospy.me1 import ME_NLP1

me_nlp = ME_NLP1(me, growth_key='mu')
muopt, hs, xopt, cache = me_nlp.bisectmu(precision=1e-6, mumax=1.5)
me.solution.f = me.solution.x_dict['biomass_dilution']

In [ ]:
from cobrame.solve.algorithms import binary_search

binary_search(me, min_mu=.1, max_mu=1.5, debug=True, mu_accuracy=1e-6)